In [11]:
# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Visualization imports
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Datetime imports
from datetime import datetime

# Stats and autocorrelation tools
import statsmodels.api
from statsmodels.tsa.stattools import acf

In [2]:
# Helper functions that use for EDA
def get_info_and_stats(df):
    '''
    get_info_stats takes a DataFrame and returns information
    about the content of the DataFrame, the number of null
    values and a statistical description of the DataFrame
    '''
    print(df.info(), df.isna().mean(), df.describe(), sep='\n#############################################\n')

def multi_frequency(df,vars):
    '''
    multi_frequency takes a dataframe and a list of columns,
    return a dataframe with the count and the frequency of the missing data
    '''
    frequency=df[vars].isnull().sum()
    percentage=df[vars].isnull().sum()*100/(len(df))
    df=pd.concat([frequency,percentage], axis=1, keys=['num_rows_missing', 'pct_rows_missing'])
    return df

def value_counts(df):
    cat_cols = df.columns[[df[col].dtype == 'O' for col in df.columns]]
    for col in cat_cols:
        print(df[col].value_counts())
        print(df[col].value_counts(normalize=True, dropna=False))
        print('\n###################################\n')

In [10]:
def plot_continuous_distribution(data: pd.DataFrame = None, column: str = None, height: int = 12):
  _ = sns.displot(data, x=column, kde=True, height=height, aspect=height/5,
                  palette='colorblind').set(title=f'Distribution of {column}');

def get_unique_values(data, column):
  num_unique_values = len(data[column].unique())
  value_counts = data[column].value_counts()
  print(f"Column: {column} has {num_unique_values} unique values\n")
  print(value_counts, '\n')

def plot_categorical_distribution(data: pd.DataFrame = None, column: str = None, height: int = 8, aspect: int = 2):
  plot = sns.catplot(
      data=data,
      x=column, 
      kind='count', 
      height=height, aspect=aspect, palette='colorblind').set(title=f'Distribution of {column}')
  plot.set_xticklabels(rotation=90)

# This is the code from the Cognizant project 1st task
def correlation_plot(data: pd.DataFrame = None):
  corr = df.corr()
  corr.style.background_gradient(cmap='coolwarm')

# This is the same code that I modified to plot a triangular 
# heatmap (without doubling/repeat the results)

def correlation_plot(data: pd.DataFrame = None):
  corr = df.corr()
  f, ax = plt.subplots(figsize=(11, 9))
  # Mask for the values of the heatmap that repeat themselves
  mask = np.triu(np.ones_like(corr, dtype=bool))
  # Creating a diverging colormap
  cmap = sns.diverging_palette(200, 20, as_cmap=True)
  sns.heatmap(corr,  mask=mask, cmap=cmap, vmax=.3, center=0,
              square=True, linewidths=.5, cbar_kws={"shrink": .5})

'''
I use parts of the code below to make the
modification in the previous function above
'''
#     sns.set_theme(style="white")
#     # Generate a large random dataset
#     rs = np.random.RandomState(33)
#     d = pd.DataFrame(data=rs.normal(size=(100, 26)),
#                      columns=list(ascii_letters[26:]))
#     # Compute the correlation matrix
#     corr = d.corr()
#     # Generate a mask for the upper triangle
#     mask = np.triu(np.ones_like(corr, dtype=bool))=
#     # Set up the matplotlib figure
#     f, ax = plt.subplots(figsize=(11, 9))
#     # Generate a custom diverging colormap
#     cmap = sns.diverging_palette(230, 20, as_cmap=True)
#     # Draw the heatmap with the mask and correct aspect ratio
#     sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
#                 square=True, linewidths=.5, cbar_kws={"shrink": .5})


'''
This is an AUTOcorrelation plot function
'''
# (not a correlation plot function)

def auto_corr_plot(series, plot_pacf=False):
    '''
    using plotly express to plot an autocorrelation plot
    plot_pacf=is set at False to avoid using a 
    partial autocorrelation function
    '''
#     To add pacf use 'if plot_pacf else acf(series.dropna(), alpha=0.05)'
#   after the line below and make needed adjustment in the function
#   (plot_pacf=True for instance)
    corr_array = acf(series.dropna(), alpha=0.05)
    lower_y = corr_array[1][:,0] - corr_array[0]
    upper_y = corr_array[1][:,1] - corr_array[0]
# Setting up the plot in plotly and the theme and shape of the plot
    fig = go.Figure()
    [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') 
     for x in range(len(corr_array[0]))]
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4',
                   marker_size=12)
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
    fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
            fill='tonexty', line_color='rgba(255,255,255,0)')
    fig.update_traces(showlegend=False)
    fig.update_xaxes(range=[-1,42])
    fig.update_yaxes(zerolinecolor='#000000')
#   Setting up the title
    title='Autocorrelation (ACF)'
    fig.update_layout(title=title)
    fig.show()
    
    
# Function to convert a timestamp with no specific frequency (it has datetime(ns)),
# and then use strftime (setting up a new format specifier) and finally strptime
# to Parse the string into a datetime object with the same format

# Needs to be transformed from the code source to something more useful for me
def convert_timestamp_to_hourly(data: pd.DataFrame = None, column: str = None):
  dummy = data.copy()
  new_ts = dummy[column].tolist()
  new_ts = [i.strftime('%Y-%m-%d %H:00:00') for i in new_ts]
  new_ts = [datetime.strptime(i, '%Y-%m-%d %H:00:00') for i in new_ts]
  dummy[column] = new_ts
  return dummy

In [15]:
# Trying to make this function work from the one right above
def get_hourly_freq(df, column):
    col = df[column].tolist()
    for i in col:
        i = col.strftime('%Y-%m-%d %H:00:00')
        i = datetime.strptime(i, '%Y-%m-%d %H:00:00')
        
        
# This ☝🏿 has become this 👇🏿 but I don't know if it's gonna work yet
def get_hourly_freq(df, column):
    col = df[column].tolist()
    for i in col:
        col = i.strftime('%Y-%m-%d %H:00:00')
        col = datetime.strptime(i, '%Y-%m-%d %H:00:00')
        df[column] = col
    return df


In [14]:
# # Plotting with seaborn and rotating the x axis

# sns.set(rc={'figure.figsize':(17,10)})
# chart = sns.histplot(
#     data=df,
#     x='category',
#     palette='colorblind',
#     hue='category',
# )
# var = chart.set_xticklabels(chart.get_xticklabels(), rotation=45)

In [16]:
# Trying to make this function work from the one right above
def get_hourly_freq(df, column):
    col = df[column].tolist()
    for i in col:
        i = col.strftime('%Y-%m-%d %H:00:00')
        i = datetime.strptime(i, '%Y-%m-%d %H:00:00')
        
        
# This ☝🏿 has become this 👇🏿 but I don't know if it's gonna work yet
def get_hourly_freq(df, column):
    col = df[column].tolist()
    for i in col:
        col = i.strftime('%Y-%m-%d %H:00:00')
        col = datetime.strptime(i, '%Y-%m-%d %H:00:00')
        df[column] = col
    return df
